In [1]:
import pandas as pd

import pickle
import numpy as np

import requests
from concurrent.futures import ThreadPoolExecutor, as_completed

from unidecode import unidecode

***
***

## API CEP -> endereço
Confirma somente que a rua está relacionada com o CEP (não checa número -> verificador do CEP)

In [2]:
dfs = []
for i in range(1,7):
    df_aux = pd.DataFrame(columns = ['Logradouro', 'Complemento'])
    df_aux.loc[-1] = ['a', 'a']

    with open(f'../results/cep_batch_{i}.pkl', 'rb') as file:
        loaded = pickle.load(file)
    
    for k, v in loaded.items():
        df_aux.loc[k] = [str(d) for d in v.tolist()]
    
    # Replace '' with NaN in the specified columns
    df_aux[['Logradouro', 'Complemento']] = df_aux[['Logradouro', 'Complemento']].replace('', np.nan)
    
    # df_aux = df_aux.dropna(how = 'all')

    df_aux.drop(-1, inplace = True)
    
    dfs.append(df_aux)
    
df_cep = pd.concat(dfs)

df_cep = df_cep.reset_index().rename(columns = {'index' : 'CEP'})
df_cep['CEP'] = df_cep['CEP'].astype(str)


df_cep = df_cep.sort_values(by='Logradouro', na_position='last')
df_cep = df_cep.drop_duplicates(subset='CEP', keep='first')

df_cep['Status'] = df_cep[['Logradouro', 'Complemento']].notnull().any(axis = 1)


print('CEPs verificados:', df_cep.shape[0])


df_cep_checked = df_cep[df_cep['Status'] == True].copy() #.drop(columns = 'Status').copy()
print('CEPs confirmados:', df_cep_checked.shape[0])

df_cep_checked[['TIPO_RUA', 'DESC_RUA']] = df_cep_checked['Logradouro'].str.split(' ', n = 1, expand = True)

df_cep_checked['DESC_RUA'] = df_cep_checked['DESC_RUA'].str.upper().apply(unidecode)
df_cep_checked = df_cep_checked.drop(columns = ['Logradouro', 'TIPO_RUA'])

df_cep_checked = df_cep_checked[['CEP', 'DESC_RUA', 'Complemento', 'Status']]
df_cep_checked.head()

CEPs verificados: 50395
CEPs confirmados: 43625


,CEP,DESC_RUA,Complemento,Status
11194,08270155,TRAVESSA MARIA MAYER,NaN,True
5551,04864230,ANDORINHA CRISTAL,NaN,True
15108,04864200,ANUM-PRETO,NaN,True
10827,04864220,ARARA AZUL,NaN,True
13126,04864180,BEM-TE-VI AMARELO,NaN,True


## Scrap Endereço -> CEP

Dada rua, número (e eventualmente nome do empreendimento) retorna o CEP.

(Para acrescentar CEP na tabela de dimensonamento das ligações)

In [3]:
dfs = []
for i in range(1,10):
    with open(f'../results/results_web_batch_{i}.pkl', 'rb') as file:
        loaded = pickle.load(file)

    df_aux = pd.DataFrame(loaded, columns=['col1', 'col2', 'col3', 'col4'])

    dfs.append(df_aux)

df_scrap = pd.concat(dfs)

df_scrap.drop(columns=['col1', 'col2'], inplace=True)
df_scrap.rename(columns = {'col3' : 'Logradouro',
                         'col4' : 'CEP'}, inplace=True)


df_scrap['teste'] = df_scrap['Logradouro'] + '__ ' + df_scrap['CEP'] 

df_scrap[['lugar', 'CEP_0']] =  df_scrap['teste'].str.split('São Paulo - SP,', n = 1, expand = True)

df_scrap = df_scrap.reset_index(drop = True).reset_index().rename(columns = {'index' : 'Id'})


scrap_ceps = df_scrap[~df_scrap['CEP_0'].isna()].copy()
scrap_ceps['CEP'] = scrap_ceps['CEP'].str.replace('São Paulo - SP, ', '')

scrap_ceps[['Logradouro', 'Bairro']] = scrap_ceps['Logradouro'].str.split(' - ', n = 1, expand = True)
scrap_ceps[['TIPO_RUA', 'DESC_RUA']] = scrap_ceps['Logradouro'].str.split(' ', n = 1, expand = True)
# scrap_ceps['DESC_RUA'] = scrap_ceps['DESC_RUA'].str.upper().str.replace('.', '').apply(unidecode)

# data_CG_SP = data_CG_SP[~data_CG_SP['DESC_RUA'].isna()]
scrap_ceps.reset_index(inplace = True)


probs_logCEP = scrap_ceps[scrap_ceps['TIPO_RUA'].isin(['Estacionamento', 'Edifício', 'Auto', 'Goldlyne', 'B',  'Vicente',
                                              'Eztec-LB-Chucri', 'Terreno', 'Residencial', 'Gran', 'E', 'Metrô',
                                              'Torre', 'Fit', 'Espaço', 'EDIFÍCIO', 'Plano', 'Raposo', 'UPA'])]
check_indexess = probs_logCEP.index



tipo_rua_values = ['Estacionamento', 'Edifício', 'Auto', 'Goldlyne', 'B',  'Vicente', 'Eztec-LB-Chucri', 
                   'Terreno', 'Residencial', 'Gran', 'E', 'Metrô', 'Torre', 'Fit', 'Espaço', 'EDIFÍCIO', 
                   'Plano', 'Raposo', 'UPA']

scrap_ceps.loc[scrap_ceps['TIPO_RUA'].isin(tipo_rua_values), 'Logradouro'] = scrap_ceps['CEP']
scrap_ceps['Logradouro'] = scrap_ceps['Logradouro'].str.replace(r', \d{8}$', '', regex=True)
scrap_ceps[['Logradouro', 'Bairro']] = scrap_ceps['Logradouro'].str.split(' - ', n = 1, expand = True)
scrap_ceps[['TIPO_RUA', 'DESC_RUA']] = scrap_ceps['Logradouro'].str.split(' ', n = 1, expand = True)
scrap_ceps['DESC_RUA'] = scrap_ceps['DESC_RUA'].str.upper().str.replace('.', '').apply(unidecode)
scrap_ceps[['DESC_RUA', 'NUMERO_RUA']] = scrap_ceps['DESC_RUA'].str.split(', ', n = 1, expand = True)

# scrap_ceps['NUMERO_RUA']  = scrap_ceps['NUMERO_RUA'].astype(float)

scrap_ceps['CEP_0'] = scrap_ceps['CEP_0'].str.replace('-', '')
# df_scrap_ceps.loc[check_indexess]

df_scrap_ceps = scrap_ceps[['Id', 'TIPO_RUA', 'DESC_RUA', 'NUMERO_RUA', 'CEP_0']].rename(columns = {'CEP_0' : 'CEP'}).drop_duplicates().dropna(how = 'all')

print('Total de endereços consultados:\t', len(df_scrap_ceps))
df_scrap_ceps.head()

Total de endereços consultados:	 6559


,Id,TIPO_RUA,DESC_RUA,NUMERO_RUA,CEP
0,0,R.,RITA JOANA DE SOUSA,464,04601061
1,1,Av.,LUIS STAMATIS,1000,02260001
2,2,Av.,VER JOSE DINIZ,2275,04603001
3,3,R.,JOAO JOSE RODRIGUES,None,08070500
4,4,R.,SUL,74,04601028


# Dados ligação

In [13]:
data_ligacao = pd.read_excel('../data/Dimensionamento_ligacao.xlsx')
print('Tamanho inicial:\t\t\t\t', data_ligacao.shape[0])

data_ligacao_SP = data_ligacao[data_ligacao['NOM_MUN'] == 'SAO PAULO'][['nom_emp', 'end_emp', 'NOM_MUN']].drop_duplicates()

data_ligacao_SP['end_emp'] = data_ligacao_SP['end_emp'].str.upper().str.replace('.', '').apply(unidecode).str.replace(' No ', '')
data_ligacao_SP['end_emp'] = data_ligacao_SP['end_emp'].str.replace(' - SAO PAULO - SP', '').str.replace('/SP','').str.replace(' - SAO PAULO', '')

data_ligacao_SP['nome_clean'] = data_ligacao_SP['nom_emp'].str.upper().str.replace('.', '').apply(unidecode).fillna('')

data_ligacao_SP['search_query'] = data_ligacao_SP['end_emp'] + '  ' + data_ligacao_SP['nome_clean']
data_ligacao_SP = data_ligacao_SP.reset_index(drop = True).reset_index().rename(columns = {'index' : 'Id'})

print('Tamanho final (capital):\t\t', data_ligacao_SP.shape[0])


df_ligacoes = pd.merge(data_ligacao_SP[['Id', 'nome_clean', 'end_emp']], df_scrap_ceps, how = 'left', on ='Id')

df_ligacoes_clean = df_ligacoes[~df_ligacoes['CEP'].isna()].copy()
df_ligacoes_clean['NUMERO_RUA'] = df_ligacoes_clean['NUMERO_RUA'].str.extract(r'(\d+)').astype(float)
df_ligacoes_clean['CEP'] = df_ligacoes_clean['CEP'].str.strip()

print('Endereços únicos (capital):\t\t', df_ligacoes_clean[['DESC_RUA', 'NUMERO_RUA', 'CEP']].dropna().drop_duplicates().shape[0])

df_ligacoes_clean.head()

Tamanho inicial:				 18023
Tamanho final (capital):		 8497
Endereços únicos (capital):		 4904


,Id,nome_clean,end_emp,TIPO_RUA,DESC_RUA,NUMERO_RUA,CEP
0,0,EDIFICIO DE USO MISTO PRACTICAL LIFE CAMPO BE...,"RUA RITA JOANA DE SOUZA, 464_CAMPO BELO",R.,RITA JOANA DE SOUSA,464.0,04601061
1,1,CONDOMINIO RESIDENCIAL MELODIA JACANA,"AV LUIS SATAMATIS, 1000 - JACANA",Av.,LUIS STAMATIS,1000.0,02260001
2,2,EDIFICIO RESIDENCIAL SOLIDI,"AV VEREADOR JOSE DINIZ,2275 - CAMPO BELO",Av.,VER JOSE DINIZ,2275.0,04603001
3,3,GALPOES PARA DEPOSITO,RUA JOAO JOSE RODRIGUES,R.,JOAO JOSE RODRIGUES,NaN,08070500
4,4,INSIGNIA CAMPO BELO - RESIDENCIAL,"RUA SUL,74 - CAMPO BELO",R.,SUL,74.0,04601028


***
***

# Arquivos:

## Coordenadas_BOS
file_path = 'data/COORDENADAS_BOS_JUNHO_2024.csv'

In [4]:
data_SP = pd.read_csv('../data/coordenadas_SP.csv')

print('Tamanho inicial (capital):\t\t', data_SP.shape[0])

id_coord = []
id_coord.append(data_SP.shape[0])

data_SP['CEP'] = data_SP['CEP'].fillna(19999991).astype(int).astype(str)
data_SP['CEP'] = data_SP['CEP'].apply(lambda x: x.zfill(8))


# data_SP['CEP'] = data_SP['CEP'].apply(
#     lambda x: x.zfill(8)[:5] + '-' + x.zfill(8)[5:] if len(x) == 7 else x[:5] + '-' + x[5:]
# )

data_SP['PDE'] = data_SP['PDE'].astype(str).apply(
    lambda x: x.zfill(10)
)

#unica rua com essa formatação
data_SP['DESC_RUA'] = data_SP['DESC_RUA'].str.replace('25 DE', 'VINTE E CINCO DE')
data_SP = data_SP[(data_SP['CEP'] != '11111-111') & (~data_SP['COORD_Y'].isna())]

data_SP.drop(columns = ['Unnamed: 0', 'DATA_LEITURA', 'COMPLEMENTO'], inplace = True)

data_SP['LAT'] = data_SP['LAT'].astype(str).str.replace(',', '.').astype(float).round(4)
data_SP['LONG'] = data_SP['LONG'].astype(str).str.replace(',', '.').astype(float).round(4)

data_SP = data_SP.drop_duplicates()
print('Tamanho final (s/ duplicados):  ', data_SP.shape[0])
# print('Endereços únicos (capital):\t\t', data_SP[['TIPO_RUA', 'DESC_RUA', 'NUMERO_RUA', 'LAT', 'LONG']].drop_duplicates().shape[0])

id_coord.append(data_SP.shape[0])

# data_SP.head()

Tamanho inicial (capital):		 3279446
Tamanho final (s/ duplicados):   2616106


In [32]:
data_SP_checked = pd.merge(data_SP, df_cep_checked, how = 'left', on = ['DESC_RUA', 'CEP'])
data_SP_checked['Status'] = data_SP_checked['Status'].fillna(False)

data_SP_checked = data_SP_checked[data_SP_checked['Status'] == True]

# data_SP_checked = data_SP[data_SP['CEP'].isin(df_cep['CEP'])].copy()

print('Endereços (capital):\t\t\t', data_SP.shape[0])
print('Endereços CEP confirmado:\t\t', data_SP_checked.shape[0])
print('\nPDEs únicos:\t\t\t\t\t', data_SP_checked['PDE'].unique().shape[0])
print('\tEndereço com PDE único:\t\t', sum(data_SP_checked[['DESC_RUA', 'NUMERO_RUA', 'CEP', 'PDE']].groupby(['DESC_RUA', 'NUMERO_RUA', 'CEP',])['PDE'].count() == 1))
print('\tEndereços com vários PDEs:\t ', sum(data_SP_checked[['DESC_RUA', 'NUMERO_RUA', 'CEP', 'PDE']].groupby(['DESC_RUA', 'NUMERO_RUA', 'CEP',])['PDE'].count() > 1))
print('\t\tPDEs "agrupados":\t\t', (data_SP_checked['PDE'].unique().shape[0] - sum(data_SP_checked[['DESC_RUA', 'NUMERO_RUA', 'CEP', 'PDE']].groupby(['DESC_RUA', 'NUMERO_RUA', 'CEP',])['PDE'].count() == 1)))

data_SP_checked.head()

Endereços (capital):			 2616106
Endereços CEP confirmado:		 2056232

PDEs únicos:					 2052892
	Endereço com PDE único:		 1023422
	Endereços com vários PDEs:	  343797
		PDEs "agrupados":		 1029470


,PDE,CODE_ENTIDADE,DESC_ENTIDADE,ATC,CEP,MUNICIPIO,TIPO_RUA,DESC_RUA,NUMERO_RUA,COORD_Y,LAT,LONG,Complemento,Status
0,2000478360,1027,OCSM,919,03385100,SAO PAULO,RUA,LUIS MARIN DOS SANTOS,87.0,52511007.0,-23.5840,-46.5251,NaN,True
1,0746231849,1027,OCSM,919,03385060,SAO PAULO,RUA,LAZARO GONCALVES FRAGA,58.0,5254766.0,-23.5829,-46.5255,NaN,True
2,0929391632,1027,OCSM,919,03385050,SAO PAULO,RUA,SERAFIM DE ABREU,8.0,5258927.0,-23.5844,-46.5259,NaN,True
5,0806690267,1027,OCSM,919,03385090,SAO PAULO,RUA,AMATAXIRO,71.0,524366.0,-23.5834,-46.5244,NaN,True
6,0125047851,1027,OCSM,919,03385050,SAO PAULO,RUA,SERAFIM DE ABREU,79.0,5256699.0,-23.5840,-46.5257,NaN,True


***
***
***

## File: CargaDados (GeoEmbraesp)

In [11]:
data_CG = pd.read_csv('../data/CargaDados_ahr.csv', delimiter = ';',  encoding='latin1', dtype = str)


data_CG['Cidade'] = data_CG['Cidade'].str.upper().apply(unidecode)

data_CG_SP = data_CG[data_CG['Cidade'] == 'SAO PAULO'].copy()

data_CG_SP[['TIPO_RUA', 'DESC_RUA']] = data_CG_SP['Logradouro'].str.split(' ', n = 1, expand = True)
data_CG_SP = data_CG_SP[~data_CG_SP['DESC_RUA'].isna()]
data_CG_SP['DESC_RUA'] = data_CG_SP['DESC_RUA'].str.upper().apply(unidecode)

data_CG_SP['LAT'] = data_CG_SP['Latitude'].str.replace(',', '.').astype(float).round(4)
data_CG_SP['LONG'] = data_CG_SP['Longitude'].str.replace(',', '.').astype(float).round(4)

# data_CG_SP = data_CG_SP[['guId', 'gufId', 'NomeEmpreendimento', 'Lancamento', 'Logradouro', 'Numero', 'Bairro',
#        'Cidade', 'CEP', 'Zona_de_Valor', 'Zoneamento', 'Setor', 'Quadra', 'Lote', 'TIPO_RUA', 'DESC_RUA', 'LAT', 'LONG']]

data_CG_SP['CEP'] = data_CG_SP['CEP'].fillna('-1')
data_CG_SP['CEP'] = data_CG_SP['CEP'].str.replace('-', '')

data_CG_SP.rename(columns = {'Numero' : 'NUMERO_RUA'}, inplace = True)

data_CG_SP['NUMERO_RUA'] = data_CG_SP['NUMERO_RUA'].fillna('9999999')
data_CG_SP = data_CG_SP[~data_CG_SP['NUMERO_RUA'].str.contains(r'[^0-9]')]

data_CG_SP['NUMERO_RUA'] = data_CG_SP['NUMERO_RUA'].astype(float)

data_CG_SP = data_CG_SP.drop_duplicates()

number_mapping = [
    ['PRIMEIRO DE', '1 DE'],
    ['SEGUNDO DE', '2 DE'],
    ['TRES DE', '3 DE'],
    ['QUATRO DE', '4 DE'],
    ['CINCO DE', '5 DE'],
    ['SEIS DE', '6 DE'],
    ['SETE DE', '7 DE'],
    ['OITO DE', '8 DE'],
    ['NOVE DE', '9 DE'],
    ['DEZ DE', '10 DE'],
    ['ONZE DE', '11 DE'],
    ['DOZE DE', '12 DE'],
    ['TREZE DE', '13 DE'],
    ['CATORZE DE', '14 DE'],
    ['QUINZE DE', '15 DE'],
    ['DEZESSEIS DE', '16 DE'],
    ['DEZESSETE DE', '17 DE'],
    ['DEZOITO DE', '18 DE'],
    ['DEZENOVE DE', '19 DE'],
    ['VINTE DE', '20 DE'],
    ['VINTE E UM DE', '21 DE'],
    ['VINTE E DOIS DE', '22 DE'],
    ['VINTE E TRÊS DE', '23 DE'],
    ['VINTE E QUATRO DE', '24 DE'],
    ['VINTE E CINCO DE', '25 DE'],
    ['VINTE E SEIS DE', '26 DE'],
    ['VINTE E SETE DE', '27 DE'],
    ['VINTE E OITO DE', '28 DE'],
    ['VINTE E NOVE DE', '29 DE'],
    ['TRINTA DE', '30 DE'],
    ['TRINTA E UM DE', '31 DE'],]

# Apply the function with the mapping argument
for day in number_mapping[::-1]:
    data_CG_SP['DESC_RUA'] = data_CG_SP['DESC_RUA'].str.strip().str.replace(str(day[1]), str(day[0])).copy()


print('Tamanho inicial:\t\t\t\t', data_CG.iloc[:, :15].drop_duplicates().shape[0])    
print('Tamanho final (capital):\t\t', data_CG_SP.iloc[:, :15].drop_duplicates().shape[0])

print('\n' + 10 * '**//**' + '\n')

# Merges
data_CG_SP.loc[(data_CG_SP['DESC_RUA'].str.contains('ANGA')) & (data_CG_SP['NUMERO_RUA'] == 894), 'NUMERO_RUA'] = 896

merge_CG_LATLONG = pd.merge(data_CG_SP,
               data_SP_checked[['PDE', 'CODE_ENTIDADE', 'CEP', 'TIPO_RUA', 'DESC_RUA', 'NUMERO_RUA', 'LAT', 'LONG', 'Status']],
               how = 'inner',
               on = ['LAT', 'LONG'],
               suffixes=('', '_DROP'))

merge_CG_LOGRADOURO = pd.merge(data_CG_SP,
               data_SP_checked[['PDE', 'CODE_ENTIDADE', 'CEP', 'TIPO_RUA', 'DESC_RUA', 'NUMERO_RUA', 'LAT', 'LONG', 'Status']],
               how = 'inner',
               on = ['CEP', 'DESC_RUA', 'NUMERO_RUA'],
               suffixes=('', '_DROP'))

merge_CG_RUA_NUM = pd.merge(data_CG_SP,
               data_SP_checked[['PDE', 'CODE_ENTIDADE', 'CEP', 'TIPO_RUA', 'DESC_RUA', 'NUMERO_RUA', 'LAT', 'LONG', 'Status']],
               how = 'inner',
               on = ['DESC_RUA', 'NUMERO_RUA'],
               suffixes=('', '_DROP'))

df_CG_PDE = pd.concat([merge_CG_LATLONG, merge_CG_LOGRADOURO], axis = 0)
df_CG_PDE.drop(columns = [c for c in df_CG_PDE.columns if '_DROP' in c], inplace = True)


print(f'Merge dados "coordenadas" x "carga dados" -> ligar PDE com endereço')
print(f'{"Size merge LATLONG:" :<25}{merge_CG_LATLONG.iloc[:, :15].drop_duplicates().shape[0]} ({merge_CG_LATLONG.iloc[:, 2:18].drop_duplicates().shape[0]})')
print('\nConsiderando o endereço:')
print(f'{"Size merge LOGRADOURO:" :<25}{merge_CG_LOGRADOURO.iloc[:, :15].drop_duplicates().shape[0]} ({merge_CG_LOGRADOURO.iloc[:, 2:18].drop_duplicates().shape[0]})')
print(f'\t{"Size merge TOTAL:" :<21}{df_CG_PDE.iloc[:, :15].drop_duplicates().shape[0]} ({df_CG_PDE.iloc[:, 2:18].drop_duplicates().shape[0]})')
print('\nALTERNATIVAMENTE:')
print(f'{"Size merge RUA_NUMERO:" :<25}{merge_CG_RUA_NUM.iloc[:, :15].drop_duplicates().shape[0]} ({merge_CG_RUA_NUM.iloc[:, 2:18].drop_duplicates().shape[0]})')
print(f'\t{"Size merge TOTAL:" :<21}{pd.concat([merge_CG_LATLONG, merge_CG_RUA_NUM], axis = 0).iloc[:, :15].drop_duplicates().shape[0]} ({pd.concat([merge_CG_LATLONG, merge_CG_RUA_NUM], axis = 0).iloc[:, 2:18].drop_duplicates().shape[0]})')

df_CG_PDE.head()

Tamanho inicial:				 24889
Tamanho final (capital):		 19333

**//****//****//****//****//****//****//****//****//****//**

Merge dados "coordenadas" x "carga dados" -> ligar PDE com endereço
Size merge LATLONG:      3356 (3340)

Considerando o endereço:
Size merge LOGRADOURO:   4564 (4540)
	Size merge TOTAL:    6779 (6737)

ALTERNATIVAMENTE:
Size merge RUA_NUMERO:   8333 (8274)
	Size merge TOTAL:    9697 (9623)


,guId,gufId,NomeEmpreendimento,NomeEmpreendimentoF,Tipo_imovel,Tipo_empreendimento,Ficha,Blocos,Unidades_andar,Andares_tipo,...,Latitude_1,Longitude_1,Observacoes,TIPO_RUA,DESC_RUA,LAT,LONG,PDE,CODE_ENTIDADE,Status
0,656,1252,RESIDENCIAL DAS FLORES,NaN,R,H,FICHA A,NaN,NaN,2.00,...,"23º 28' 23,62'' S","46º 41' 29,51'' W",TRATA-SE DE CONDOMÍNIO HORIZONTAL.,RUA,JOSE DE CAMPOS NOVAIS,-23.4732,-46.6915,0167668811,1130,True
1,658,1255,GREEN SOLARIUM RESIDENCE,NaN,R,V,FICHA A,0.85,4.00,13.00,...,"23º 29' 5,24'' S","46º 42' 4,95'' W","UNIDADES-TIPO DIFERENCIADAS, VIDE FICHA B. BLO...",RUA,RIO VERDE,-23.4848,-46.7014,0171297024,1130,True
2,658,1256,GREEN SOLARIUM RESIDENCE,NaN,R,V,FICHA B,0.15,4.00,13.00,...,"23º 29' 5,24'' S","46º 42' 4,95'' W","UNIDADES-TIPO DIFERENCIADAS, VIDE FICHA A. BLO...",RUA,RIO VERDE,-23.4848,-46.7014,0171297024,1130,True
3,667,1270,CONDOMÍNIO CAMPO DAS PITANGUEIRAS,NaN,R,H,FICHA A,NaN,NaN,2.00,...,"23º 31' 23,03'' S","46º 28' 40,57'' W",TRATA-SE DE CONDOMÍNIO HORIZONTAL.,RUA,CAMPO DAS PITANGUEIRAS,-23.5231,-46.4779,0471419486,1103,True
4,679,1286,INSPIRE JARDIM PENHA,NaN,R,V,FICHA A,1.00,5.78,9.00,...,"23º 30' 23,89'' S","46º 28' 25,7'' W",UNID/AND: 4 UNID NO TÉRREO E 6 DO 1º AO 8º ANDAR.,RUA,CORONEL RODOLFO PORTO,-23.5066,-46.4738,0137502087,1103,True


In [44]:
zxc = df_CG_PDE[(df_CG_PDE['CEP'].str.len() != 8) | (df_CG_PDE['CEP'].isna())][['NomeEmpreendimento', 'CEP', 'DESC_RUA', 'NUMERO_RUA']].drop_duplicates()
zxc

,NomeEmpreendimento,CEP,DESC_RUA,NUMERO_RUA
1,GREEN SOLARIUM RESIDENCE,2934,RIO VERDE,280.0
6,NAO DEFINIDO,3613,ENEAS DE BARROS,769.0
15,RESIDENCIAL ALTOS DO PALANQUE,8030,PALANQUE,33.0
30,QUINTA DO MOINHO,2929,MOINHO VELHO,659.0
36,TAORMINA,3509,MARCONDES DE BRITO,1340.0
...,...,...,...,...
6808,RESIDENCIAL MANUELA SANTOS,3623,VERA,210.0
6810,CONDOMÍNIO RESIDENCIAL ROFEMABI,3623,VERA,222.0
6837,NUMBER ONE,4138,CARAMURU,1164.0
6878,I9 SAPIENZA VILA ROMANA,2618,DOMINGOS JOSE SAPIENZA,208.0


In [46]:
with open('../data_clean/PDE_empreendimentos_full.pkl', 'rb') as file:
    qwe = pickle.load(file)
    
qwe[qwe['CEP'].isin(zxc['CEP'])]

,guId,gufId,NomeEmpreendimento,NomeEmpreendimentoF,Tipo_imovel,Tipo_empreendimento,Ficha,Blocos,Unidades_andar,Andares_tipo,...,LONG,PDE,CODE_ENTIDADE,CEP_DROP,TIPO_RUA_DROP,DESC_RUA_DROP,NUMERO_RUA_DROP,Status,LAT_DROP,LONG_DROP
1,658,1255,GREEN SOLARIUM RESIDENCE,NaN,R,V,FICHA A,0.85,4.00,13.00,...,-46.7014,0171297024,1130,02934001,RUA,RIO VERDE,277.0,True,NaN,NaN
2,658,1256,GREEN SOLARIUM RESIDENCE,NaN,R,V,FICHA B,0.15,4.00,13.00,...,-46.7014,0171297024,1130,02934001,RUA,RIO VERDE,277.0,True,NaN,NaN
6,688,1301,NAO DEFINIDO,NaN,R,H,FICHA A,NaN,NaN,2.00,...,-46.5343,0729817768,1103,03613000,RUA,ENEAS DE BARROS,769.0,True,NaN,NaN
7,688,1301,NAO DEFINIDO,NaN,R,H,FICHA A,NaN,NaN,2.00,...,-46.5343,0729823903,1103,03613000,RUA,ENEAS DE BARROS,769.0,True,NaN,NaN
8,688,1301,NAO DEFINIDO,NaN,R,H,FICHA A,NaN,NaN,2.00,...,-46.5343,0783370466,1103,03613000,RUA,ENEAS DE BARROS,751.0,True,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37675,18925,33518,ESSÊNCIA PERDIZES BY SETIN,NaN,R,V,FICHA C,0.21,3.64,22.00,...,-46.6796,0109834062,1027,05018000,RUA,NaN,NaN,True,-23.5327,-46.6794
38070,19233,34185,URBIC IBIRAPUERA,NaN,R,V,FICHA A,0.33,3.00,8.00,...,-46.6516,0079687792,1027,04005004,RUA,NaN,NaN,True,-23.5783,-46.6532
38071,19233,34186,URBIC IBIRAPUERA,NaN,R,V,FICHA B,0.67,3.00,8.00,...,-46.6516,0079687792,1027,04005004,RUA,NaN,NaN,True,-23.5783,-46.6532
38093,19255,34227,MIRANTE DO HORTO,NaN,R,V,FICHA A,15.00,4.00,5.00,...,-46.6466,1043764574,1130,02636030,RUA,NaN,NaN,True,-23.4531,-46.6450


In [52]:
qwe[(qwe['CEP'].str.len() != 8) & (~qwe['CEP_DROP'].isna())][['NomeEmpreendimento', 'CEP', 'CEP_DROP', 'DESC_RUA', 'NUMERO_RUA']].drop_duplicates()

,NomeEmpreendimento,CEP,CEP_DROP,DESC_RUA,NUMERO_RUA
1,GREEN SOLARIUM RESIDENCE,2934,02934001,RIO VERDE,280.0
6,NAO DEFINIDO,3613,03613000,ENEAS DE BARROS,769.0
15,RESIDENCIAL ALTOS DO PALANQUE,8030,08030120,PALANQUE,33.0
30,QUINTA DO MOINHO,2929,02929160,MOINHO VELHO,659.0
36,TAORMINA,3509,03509000,MARCONDES DE BRITO,1340.0
...,...,...,...,...,...
38063,NaN,4551,04551060,FUNCHAL,65.0
38070,URBIC IBIRAPUERA,4005,04005004,ABILIO SOARES,1149.0
38074,SELETTO CAMPO BELO,4560,04601042,GIL EANES,635.0
38093,MIRANTE DO HORTO,2636,02636030,ITA,460.0


In [ ]:
def merge_with_tolerance(df1, df2, id_dfs, agg_cols, var_col, tolerance):
    # First, do an exact merge on the 'CEP' column
    df_merged = pd.merge(df1, df2, on = agg_cols, how = 'inner', suffixes = (id_dfs[0], id_dfs[1]))

    # Filter rows where the 'NUMERO_RUA' column from df1 is within the tolerance range of df2
    df_merged = df_merged[(df_merged[f'{var_col}' + id_dfs[0]] >= df_merged[f'{var_col}' + id_dfs[1]] - tolerance) &
                          (df_merged[f'{var_col}' + id_dfs[0]] <= df_merged[f'{var_col}' + id_dfs[1]] + tolerance)]

    # Reordering columns
    agg_col_suffixes = [f'{col}{id_dfs[0]}' for col in agg_cols] + [f'{col}{id_dfs[1]}' for col in agg_cols if
                                                                    f'{col}{id_dfs[1]}' in df_merged.columns]
    var_col_suffixes = [f'{var_col}{id_dfs[0]}', f'{var_col}{id_dfs[1]}']

    # Get the rest of the columns
    other_cols = [col for col in df_merged.columns if col not in agg_col_suffixes + var_col_suffixes]
    other_cols = [c for c in other_cols if c not in agg_cols]
    # New order: agg_cols + var_col_suffixes + other columns
    new_order = agg_cols + var_col_suffixes + other_cols

    # Reorder columns
    df_merged = df_merged[new_order]

    return df_merged

In [ ]:
def merge_df_tolerance(data_cep_iptu, data_iptu, data_imoveis, tol_NUMERO = 10, tol_PVTOS = 5, tol_OBSOLENCIA = 0.1):
    qwe = merge_with_tolerance(df1 = data_cep_iptu,
                               df2 = data_iptu[['DESC_RUA', 'NUMERO_RUA', 'CEP', 'QUANTIDADE DE PAVIMENTOS',
                                                'FATOR DE OBSOLESCENCIA', 'TIPO DE PADRAO DA CONSTRUCAO',
                                                'TESTADA PARA CALCULO', 'VALOR DO M2 DO TERRENO']],
                               id_dfs = ['_cep', '_iptu'],
                               agg_cols = ['CEP'],
                               var_col = 'NUMERO_RUA',
                               tolerance = tol_NUMERO)

    qwe['QUANTIDADE DE PAVIMENTOS'] = qwe['QUANTIDADE DE PAVIMENTOS'].astype(int)

    qwe['Imovel_residencial'] = np.where(qwe['TIPO DE PADRAO DA CONSTRUCAO'].str.contains('Residencial'), 1, 0)

    # Create 'Imovel_vertical' column: 1 if 'vertical' is in the string, 0 otherwise
    qwe['Imovel_vertical'] = np.where(qwe['TIPO DE PADRAO DA CONSTRUCAO'].str.contains('vertical'), 1, 0)

    # Create a new column for the remaining part ('padrão X')
    qwe['Padrao'] = qwe['TIPO DE PADRAO DA CONSTRUCAO'].str.extract(r'(padrão \w)')
    qwe['Padrao'] = qwe['Padrao'].str.replace('padrão ', '')

    asd = merge_with_tolerance(df1 = qwe.rename(columns = {'DESC_RUA_cep': 'DESC_RUA',
                                                           'NUMERO_RUA_cep': 'NUMERO_RUA',
                                                           'QUANTIDADE DE PAVIMENTOS': 'Andares_tipo'}),
                               df2 = data_imoveis[
                                   ['DESC_RUA', 'NUMERO_RUA', 'Imovel_residencial', 'Imovel_vertical', 'Andares_tipo',
                                    'Total_Unidades', 'Idade_predio', 'Blocos',
                                    'M2_util_unidade_tipo', 'M2_total_unidade_tipo', 'RS_por_M2_area_util_IGPM',
                                    'RS_por_M2_area_total_IGPM', ]],
                               id_dfs = ['_cep_iptu', '_cg'],
                               agg_cols = ['DESC_RUA', 'NUMERO_RUA', 'Imovel_residencial', 'Imovel_vertical'],
                               var_col = 'Andares_tipo',
                               tolerance = tol_PVTOS).drop_duplicates()

    asd = asd[asd['Imovel_vertical'] == 1]  # muitas incertezas com horizontais

    # DataFrame containing obsolescence factors
    df_factors = pd.DataFrame({
        'Idade do Prédio (em anos)': ["Menor que 1"] + list(range(1, 43)),
        'Fatores de Obsolescência para padrões A e B': [1.00, 0.99, 0.98, 0.97, 0.96, 0.94, 0.93, 0.92, 0.90, 0.89,
                                                        0.88, 0.86, 0.84, 0.83, 0.81, 0.79, 0.78, 0.76, 0.74, 0.72,
                                                        0.70, 0.68, 0.66, 0.64, 0.62, 0.59, 0.57, 0.55, 0.52, 0.50,
                                                        0.48, 0.45, 0.42, 0.40, 0.37, 0.34, 0.32, 0.29, 0.26, 0.23,
                                                        0.20, 0.20, 0.20],
        'Fatores de Obsolescência para demais padrões e tipos': [1.00, 0.99, 0.99, 0.98, 0.97, 0.96, 0.96, 0.95, 0.94,
                                                                 0.93, 0.92, 0.91, 0.90, 0.89, 0.88, 0.88, 0.86, 0.85,
                                                                 0.84, 0.83, 0.82, 0.81, 0.80, 0.79, 0.78, 0.76, 0.75,
                                                                 0.74, 0.73, 0.71, 0.70, 0.69, 0.67, 0.66, 0.64, 0.63,
                                                                 0.62, 0.60, 0.59, 0.57, 0.56, 0.54, 0.52]
        })

    def get_obsolescence_factor(age, pattern, df_factors):
        # Map age to lookup value
        if age < 1:
            age_lookup = "Menor que 1"
        else:
            age_lookup = age

        # Determine the column to use based on the pattern
        if pattern in ['A', 'B']:
            factor_column = 'Fatores de Obsolescência para padrões A e B'
        else:
            factor_column = 'Fatores de Obsolescência para demais padrões e tipos'

        # Perform the lookup and handle missing results
        factors = df_factors[df_factors['Idade do Prédio (em anos)'] == age_lookup]

        if not factors.empty:
            factor = factors[factor_column].values[0]
        else:
            # Handle the case where no matching age is found (you can adjust this as needed)
            factor = -1  # Or set a default factor value

        return factor

    # Apply the function to your DataFrame
    asd['Fator_obsolescencia_calculado'] = asd.apply(
        lambda row: get_obsolescence_factor(row['Idade_predio'], row['Padrao'], df_factors), axis = 1)

    asd = asd[abs(asd['Fator_obsolescencia_calculado'] - asd['FATOR DE OBSOLESCENCIA'].astype(float)) < tol_OBSOLENCIA]

    asd['TOL_NUM'] = abs(asd['NUMERO_RUA'] - asd['NUMERO_RUA_iptu'])

    asd['TESTADA PARA CALCULO'] = asd['TESTADA PARA CALCULO'].astype(float)

    return asd[['DESC_RUA', 'NUMERO_RUA', 'NUMERO_RUA_iptu', 'Andares_tipo_cep_iptu', 'Andares_tipo_cg', 'TOL_NUM',
                'TESTADA PARA CALCULO',
                'Fator_obsolescencia_calculado', 'FATOR DE OBSOLESCENCIA', 'Total_Unidades', 'Blocos',
                'M2_util_unidade_tipo', 'M2_total_unidade_tipo', 'RS_por_M2_area_util_IGPM',
                'RS_por_M2_area_total_IGPM', 'VALOR DO M2 DO TERRENO', 'TIPO DE PADRAO DA CONSTRUCAO']].sort_values(by = ['DESC_RUA', 'NUMERO_RUA'])


In [31]:
data_SP.head()

,PDE,CODE_ENTIDADE,DESC_ENTIDADE,ATC,CEP,MUNICIPIO,TIPO_RUA,DESC_RUA,NUMERO_RUA,COORD_Y,LAT,LONG
0,2000478360,1027,OCSM,919,03385100,SAO PAULO,RUA,LUIS MARIN DOS SANTOS,87.0,52511007.0,-23.5840,-46.5251
1,0746231849,1027,OCSM,919,03385060,SAO PAULO,RUA,LAZARO GONCALVES FRAGA,58.0,5254766.0,-23.5829,-46.5255
2,0929391632,1027,OCSM,919,03385050,SAO PAULO,RUA,SERAFIM DE ABREU,8.0,5258927.0,-23.5844,-46.5259
3,0125062575,1027,OCSM,919,03385090,SAO PAULO,RUA,ACAPORI,12.0,5252262.0,-23.5833,-46.5252
4,0604648529,1027,OCSM,919,03729100,SAO PAULO,RUA,FALANTO,68.0,52607766.0,-23.5826,-46.5261


In [33]:
data_CG

,guId,gufId,NomeEmpreendimento,NomeEmpreendimentoF,Tipo_imovel,Tipo_empreendimento,Ficha,Blocos,Unidades_andar,Andares_tipo,...,Zona_de_Valor,Zoneamento,Setor,Quadra,Lote,Latitude,Longitude,Latitude_1,Longitude_1,Observacoes
0,655,1249,MERIDIAN BARRA FUNDA,NaN,R,V,FICHA A,1.00,4.00,25.00,...,AGUA BRANCA,ZM-3A,197,18,2,"-23,5181402","-46,6654909","23º 31' 5,3'' S","46º 39' 55,77'' W","UNIDADES-TIPO DIFERENCIADAS, VIDE FICHAS B E C..."
1,655,1250,MERIDIAN BARRA FUNDA,NaN,R,V,FICHA B,0.11,4.00,25.00,...,AGUA BRANCA,ZM-3A,197,18,2,"-23,5181402","-46,6654909","23º 31' 5,3'' S","46º 39' 55,77'' W","UNIDADES-TIPO DIFERENCIADAS, VIDE FICHAS A E C..."
2,655,1251,MERIDIAN BARRA FUNDA,NaN,R,V,FICHA C,0.89,4.00,25.00,...,AGUA BRANCA,ZM-3A,197,18,2,"-23,5181402","-46,6654909","23º 31' 5,3'' S","46º 39' 55,77'' W","UNIDADES-TIPO DIFERENCIADAS, VIDE FICHAS A E B..."
3,656,1252,RESIDENCIAL DAS FLORES,NaN,R,H,FICHA A,NaN,NaN,2.00,...,BRASILÂNDIA,ZM-3B,107,224,12,"-23,4732289","-46,6915297","23º 28' 23,62'' S","46º 41' 29,51'' W",TRATA-SE DE CONDOMÍNIO HORIZONTAL.
4,657,1253,RESIDENCIAL COTTI,NaN,R,V,FICHA A,0.55,4.88,9.00,...,FREGUESIA DO Ó,ZM-3A,77,432,40,"-23,491745","-46,7164117","23º 29' 30,28'' S","46º 42' 59,08'' W",UNID/AND: 4 UNID NO TÉRREO E 5 DO 1º AO 8º AND...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24884,22601,41599,MAXIMO JARDIM,NaN,R,V,FICHA A,1.33,11.81,21.00,...,SANTO ANDRÉ,R,M,S,P,"-23,631636","-46,5373189","23º 37' 53,89'' S","46º 32' 14,35'' W",UNID/AND: 8 UNID NO 1º AND E 12 DO 2º AO 21º AND.
24885,22601,41600,MAXIMO JARDIM,NaN,R,V,FICHA B,0.67,11.81,21.00,...,SANTO ANDRÉ,R,M,S,P,"-23,631636","-46,5373189","23º 37' 53,89'' S","46º 32' 14,35'' W","UNID-TIPO DIFERENCIADAS, VIDE FICHA A. BLOCO, ..."
24886,22602,41601,NaN,HELBOR PATTEO SÃO BERNARDO,R,V,FICHA A,0.50,7.76,17.00,...,SÃO BERNARDO DO CAMPO,R,M,S,P,"-23,7072034","-46,5644849","23º 42' 25,93'' S","46º 33' 52,15'' W","EMPREEND. FORMADO POR 3 BLOCOS, DOS QUAIS 2 (A..."
24887,22602,41602,NaN,HELBOR PATTEO SÃO BERNARDO,R,V,FICHA B,0.24,7.76,17.00,...,SÃO BERNARDO DO CAMPO,R,M,S,P,"-23,7072034","-46,5644849","23º 42' 25,93'' S","46º 33' 52,15'' W","UNID-TIPO DIFERENCIADAS, VIDE FICHAS A E C. BL..."
